In [1]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt

from qiskit import Aer
from tqdm.notebook import tqdm
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4, suppress=None)

In [2]:
backend = Aer.get_backend('qasm_simulator')

# Breast Cancer Data

### Load Data

In [3]:
data = load_breast_cancer()
x = data.data
y = data.target.reshape(-1, 1)

np.random.seed(42)
x, _, y, _ = train_test_split(x, y, train_size=100)

## Three First Variables

In [4]:
x_three_first = x[:,:3]

In [5]:
y_dnn = y
y_qnn = y

x_qnn = scaler(x_three_first, mode="uniform", a=0, b=np.pi)
x_dnn = scaler(x_three_first, mode="standard")

In [6]:
np.random.seed(42)

dnn_list = []
for i in tqdm(range(10)):
    dnn = sequential_dnn(dim = [3, 5, 1], lr = 0.1)
    dnn.train(x_dnn, y_dnn, epochs = 100)
    dnn_list.append(dnn)
    
saver(dnn_list, data_path("cancer_dnn_three_first"))

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
np.random.seed(42)

qnn_list = []
for i in tqdm(range(5)):
    qnn = sequential_qnn(q_bits = [3, 3],
                         dim = [3, 3, 1],
                         reps = 2,
                         backend=backend,
                         shots=10000,
                         lr = 0.1)
    qnn.train(x_qnn, y_qnn, epochs=100, verbose=True)
    qnn_list.append(qnn)

saver(qnn_list, data_path("cancer_qnn_three_first"))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 0, loss: 0.26502782750000003
epoch: 1, loss: 0.2070092328125
epoch: 2, loss: 0.20012655734374996
epoch: 3, loss: 0.18495334703125
epoch: 4, loss: 0.15322419281250002
epoch: 5, loss: 0.16881867875
epoch: 6, loss: 0.1515238440625
epoch: 7, loss: 0.13026224453125
epoch: 8, loss: 0.11955222999999997
epoch: 9, loss: 0.11837929171875
epoch: 10, loss: 0.113347259375
epoch: 11, loss: 0.10739264921875001
epoch: 12, loss: 0.10809893015625002
epoch: 13, loss: 0.105259896875
epoch: 14, loss: 0.10158558812500001
epoch: 15, loss: 0.09745477359375002
epoch: 16, loss: 0.09503926281250001
epoch: 17, loss: 0.09591349093749998
epoch: 18, loss: 0.096543273125
epoch: 19, loss: 0.09399468843749999
epoch: 20, loss: 0.09499942749999998
epoch: 21, loss: 0.0924414909375
epoch: 22, loss: 0.0909906565625
epoch: 23, loss: 0.08952948124999999
epoch: 24, loss: 0.08944457734374998
epoch: 25, loss: 0.08889657062499999
epoch: 26, loss: 0.086397039375
epoch: 27, loss: 0.08236825249999999
epoch: 28, loss: 0.080458

## PCA

In [ ]:
x_scaled = scaler(x, mode="standard")
pca = PCA(n_components=3)
x_pca = pca.fit_transform(x_scaled)

In [ ]:
y_dnn = y
y_qnn = y

x_qnn = scaler(x_pca, mode="uniform", a=0, b=np.pi)
x_dnn = scaler(x_pca, mode="standard")

In [ ]:
plt.plot(x_qnn[:,1], y_dnn,"o")

In [ ]:
np.random.seed(42)

dnn_list = []
for i in tqdm(range(10)):
    dnn = sequential_dnn(dim = [3, 5, 1], lr = 0.1)
    dnn.train(x_dnn, y_dnn, epochs = 100)
    dnn_list.append(dnn)
    
saver(dnn_list, data_path("cancer_dnn_pca"))

In [ ]:
np.random.seed(42)

qnn_list = []
for i in tqdm(range(5)):
    qnn = sequential_qnn(q_bits = [3, 3],
                         dim = [3, 3, 1],
                         reps = 2,
                         backend=backend,
                         shots=10000,
                         lr = 0.1)
    qnn.train(x_qnn, y_qnn, epochs=100, verbose=True)
    qnn_list.append(qnn)

saver(qnn_list, data_path("cancer_qnn_pca"))

## Most Correlated

In [ ]:
corr = np.corrcoef(x, y, rowvar=False)[-1,:-1]
idx = (corr).argsort()[:3]
x_corr = x[:, idx]

In [ ]:
y_dnn = y
y_qnn = y

x_qnn = scaler(x_corr, mode="uniform", a=0, b=np.pi)
x_dnn = scaler(x_corr, mode="standard")

In [ ]:
plt.plot(x_qnn[:,2], y_dnn,"o")

In [ ]:
np.random.seed(42)

dnn_list = []
for i in tqdm(range(10)):
    dnn = sequential_dnn(dim = [3, 5, 1], lr = 0.1)
    dnn.train(x_dnn, y_dnn, epochs = 100)
    dnn_list.append(dnn)
    
saver(dnn_list, data_path("cancer_dnn_most_correlated"))

In [ ]:
np.random.seed(42)

qnn_list = []
for i in tqdm(range(5)):
    qnn = sequential_qnn(q_bits = [3, 3],
                         dim = [3, 3, 1],
                         reps = 2,
                         backend=backend,
                         shots=10000,
                         lr = 0.1)
    qnn.train(x_qnn, y_qnn, epochs=100, verbose=True)
    qnn_list.append(qnn)

saver(qnn_list, data_path("cancer_qnn"))

### Hybrid Network

### Two Layers

In [ ]:
np.random.seed(42)

network_list = []
for i in tqdm(range(5)):
    layer1 = QLayer(n_qubits=3, n_features=3, n_targets=3, encoder=Encoder(), ansatz=Ansatz(), sampler=Parity(), reps=2, scale=1, backend=backend, shots=10000)
    layer2 = Dense(n_features=3, n_targets=1, activation=Sigmoid())
    layers = [layer1, layer2]

    network = NeuralNetwork(layers=layers, optimizer = Adam(lr=0.1))
    network.train(x_qnn, y_qnn, epochs=100, verbose=True)
    network_list.append(network)

saver(network, data_path("cancer_hybrid_2_layer"))

### Three Layers

In [ ]:
np.random.seed(42)

network_list = []
for i in tqdm(range(5)):
    layer1 = QLayer(n_qubits=3, n_features=3, n_targets=3, encoder=Encoder(), ansatz=Ansatz(), sampler=Parity(), reps=2, scale=1, backend=backend, shots=10000)
    layer2 = QLayer(n_qubits=3, n_features=3, n_targets=3, encoder=Encoder(), ansatz=Ansatz(), sampler=Parity(), reps=2, scale=1, backend=backend, shots=10000)
    layer3 = Dense(n_features=3, n_targets=1, activation=Sigmoid())
    layers = [layer1, layer2, layer3]

    network = NeuralNetwork(layers=layers, optimizer = Adam(lr=0.1))
    network.train(x_qnn, y_qnn, epochs=100, verbose=True)
    network_list.append(network)

saver(network, data_path("cancer_hybrid_3_layer"))